In [ ]:
import numpy as np 
import pandas as pd 

In [ ]:
data=pd.read_csv("/kaggle/input/grocery-product-prices-for-australian-states/Australia_Grocery_2022Sep.csv")
data.drop(["Postal_code","RunDate","Product_Url","tid","state","index"], axis=1,inplace=True)

In [ ]:
df=data.dropna()
df=df.reset_index(drop=True)
df.head(2)

In [ ]:
df.head(2)

In [ ]:
import category_encoders as ce
ordenc=ce.OrdinalEncoder(cols=["unit_price_unit","package_size","Price_per_unit","Brand","Sku","city"])
df=ordenc.fit_transform(df)

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder=LabelEncoder()
df = df.apply(label_encoder.fit_transform)

In [ ]:
df.head(2)

# PCA

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
principal_components = pca.fit_transform(df)
component_names = [f"Component {i}" for i in range(2)]
pca_data = pd.DataFrame(principal_components, columns=component_names)
pca_data.head(2)

In [ ]:
pca.explained_variance_ratio_

****

****

# KMeans


In [ ]:
# Apply k-means clustering to the data
kmeans = KMeans(n_clusters=3)
kmeans.fit(pca_data)

# Assign each data point to a cluster
clusters = kmeans.predict(pca_data)
pc=principal_components
# Find the top 10 most similar products for each product
for i in range(len(data[:30])):
    cluster_indices = np.where(clusters == clusters[i])[0]
    print(cluster_indices)
    distances = np.linalg.norm(pc[cluster_indices] - pc[i], axis=1)
    print(distances)
    closest_indices = np.argsort(distances)[1:11]
    print(cluster_indices[closest_indices])
#     closest_products = df.loc[cluster_indices[closest_indices]]
    print("Top 10 closest products for product", i, ":", cluster_indices[closest_indices],"\n\n\n\n")

****

# DBSCAN Clustering

In [ ]:
from sklearn.cluster import DBSCAN

dbscan = DBSCAN(eps=0.5, min_samples=5)
clusters = dbscan.fit_predict(pca_data)

# Find the top 10 most similar products for each product
for i in range(len(data[:20])):
    cluster_indices = np.where(clusters == clusters[i])[0]
    distances = np.linalg.norm(pc[cluster_indices] - pc[i], axis=1)
    closest_indices = np.argsort(distances)[1:11]
#     closest_products = df.loc[cluster_indices[closest_indices]]
    print("Top 10 closest products for product", i, ":", cluster_indices[closest_indices])


****

# AgglomerativeClustering

In [ ]:
from sklearn.cluster import AgglomerativeClustering
# Apply hierarchical clustering
agg_clustering = AgglomerativeClustering(n_clusters=5)
agg_clustering.fit(pca_data)

# Find the top 10 most similar products for each product
for i in range(len(df[:20])):
    cluster_idx = agg_clustering.labels_[i]
    similar_products = np.where(agg_clustering.labels_ == cluster_idx)[0]
    top_10_similar_products = similar_products[:10]
    print("Product", i, "is most similar to:", top_10_similar_products)


****

****

# DBSCAN with cosine_similarity

In [ ]:
from sklearn.cluster import DBSCAN
from scipy.spatial import distance
from sklearn.metrics.pairwise import cosine_similarity



dbscan = DBSCAN(eps=0.5, min_samples=5)
clusters = dbscan.fit_predict(pca_data)

for i in range(len(data[:20])):
    cluster_indices = np.where(clusters == clusters[i])[0]
    p1 = np.array(pc[cluster_indices]).reshape(-1, pc.shape[1])
    p2 = np.array(pc[i]).reshape(1, -1)
    similarities = cosine_similarity(p1, p2)
    closest_indices = np.argsort(similarities.flatten())[-10:]
    print("Top 10 closest products for product", i, ":", cluster_indices[closest_indices])



****